In [5]:
# Ohio Low Birth Weight Analysis
# Analysis of birth outcomes across Ohio counties (2020-2025)
install.packages("lme4")
install.packages("broom.mixed")

# Load required libraries
library(tidyverse)
library(lme4)        # For mixed effects models
library(ggplot2)
library(viridis)
library(gridExtra)
library(broom.mixed) # For tidy model output
library(scales)

Warning message:
"package 'lme4' is in use and will not be installed"
Installing package into 'C:/Users/kehin/AppData/Local/R/win-library/4.5'
(as 'lib' is unspecified)

also installing the dependencies 'coda', 'furrr'




package 'coda' successfully unpacked and MD5 sums checked
package 'furrr' successfully unpacked and MD5 sums checked
package 'broom.mixed' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\kehin\AppData\Local\Temp\RtmpeckKP7\downloaded_packages



Attaching package: 'scales'


The following object is masked from 'package:viridis':

    viridis_pal


The following object is masked from 'package:purrr':

    discard


The following object is masked from 'package:readr':

    col_factor




In [7]:
# Set theme for all plots
theme_set(theme_minimal() + 
  theme(plot.title = element_text(face = "bold", size = 14),
        plot.subtitle = element_text(size = 10),
        axis.title = element_text(size = 10)))


## DATA LOADING AND PREPARATION



In [29]:
library(dplyr)

# Remove extra index column from birth_path if it exists
if ("...1" %in% colnames(birth_path)) {
  birth_path <- birth_path %>% select(-...1)
}

# Remove extra index column from demographic if it exists
if ("...1" %in% colnames(demographic)) {
  demographic <- demographic %>% select(-...1)
}

# Inspect first few rows
head(birth_path)
head(demographic)


county_of_birth,age_of_mother,race_of_mother,low_birth_weight,birth_id
<chr>,<dbl>,<chr>,<dbl>,<dbl>
Adams County,31,White,1,1
Adams County,31,White,1,2
Adams County,31,White,1,3
Adams County,31,White,1,4
Adams County,31,White,1,5
Adams County,33,White,1,6


County,Population,Below.HS.Education.Percentage,Unemployment.Rate
<chr>,<dbl>,<dbl>,<dbl>
Adams County,27671,14.8,6.5
Allen County,100866,7.6,5.9
Ashland County,52420,9.0,3.7
Ashtabula County,96906,12.5,6.9
Athens County,63218,8.2,5.8
Auglaize County,45922,6.4,2.5


In [31]:
# Clean county names for merging (ensure consistent formatting)
birth_path <- birth_path %>%
  mutate(county_of_birth = str_trim(county_of_birth))

demographic <- demographic %>%
  mutate(County = str_trim(County))

# Merge datasets
ohio_data <- birth_path %>%
  left_join(demographic, by = c("county_of_birth" = "County"))

# Check for missing values
summary(ohio_data)

 county_of_birth    age_of_mother   race_of_mother     low_birth_weight
 Length:792526      Min.   :12.00   Length:792526      Min.   :0.0000  
 Class :character   1st Qu.:22.00   Class :character   1st Qu.:1.0000  
 Mode  :character   Median :27.00   Mode  :character   Median :1.0000  
                    Mean   :27.54                      Mean   :0.7878  
                    3rd Qu.:32.00                      3rd Qu.:1.0000  
                    Max.   :52.00                      Max.   :1.0000  
                    NA's   :24                                         
    birth_id         Population      Below.HS.Education.Percentage
 Min.   :      1   Min.   :  12545   Min.   : 2.900               
 1st Qu.: 249865   1st Qu.:  64692   1st Qu.: 7.100               
 Median : 495607   Median : 172347   Median : 8.500               
 Mean   : 504717   Mean   : 369813   Mean   : 8.851               
 3rd Qu.: 759367   3rd Qu.: 537443   3rd Qu.: 9.000               
 Max.   :1026449   Max

In [33]:
# Create categorical age groups for visualization
ohio_data <- ohio_data %>%
  mutate(
    age_group = case_when(
      age_of_mother < 20 ~ "<20",
      age_of_mother >= 20 & age_of_mother < 25 ~ "20-24",
      age_of_mother >= 25 & age_of_mother < 30 ~ "25-29",
      age_of_mother >= 30 & age_of_mother < 35 ~ "30-34",
      age_of_mother >= 35 & age_of_mother < 40 ~ "35-39",
      age_of_mother >= 40 ~ "40+",
      TRUE ~ NA_character_
    ),
    age_group = factor(age_group, levels = c("<20", "20-24", "25-29", 
                                               "30-34", "35-39", "40+"))
  )

In [35]:
## DESCRIPTIVE STATISTICS
# Overall LBW rate
overall_lbw_rate <- mean(ohio_data$low_birth_weight, na.rm = TRUE) * 100
cat(sprintf("Overall LBW Rate: %.1f%%\n", overall_lbw_rate))

# LBW by age group
age_summary <- ohio_data %>%
  group_by(age_group) %>%
  summarize(
    n_births = n(),
    lbw_rate = mean(low_birth_weight, na.rm = TRUE) * 100,
    .groups = "drop"
  )
print(age_summary)

# LBW by race
race_summary <- ohio_data %>%
  group_by(race_of_mother) %>%
  summarise(
    n_births = n(),
    lbw_rate = mean(low_birth_weight, na.rm = TRUE) * 100,
    percentage = n() / nrow(ohio_data) * 100,
    .groups = "drop"
  ) %>%
  arrange(desc(lbw_rate))
print(race_summary)

# County-level summary
county_summary <- ohio_data %>%
  group_by(county_of_birth) %>%
  summarise(
    n_births = n(),
    lbw_rate = mean(low_birth_weight, na.rm = TRUE) * 100,
    below_hs = first(Below.HS.Education.Percentage),
    unemployment = first(Unemployment.Rate),
    population = first(Population),
    .groups = "drop"
  ) %>%
  arrange(desc(lbw_rate))

# Display highest and lowest LBW counties
cat("\nTop 5 Counties with Highest LBW Rates:\n")
print(head(county_summary, 5))
cat("\nTop 5 Counties with Lowest LBW Rates:\n")
print(tail(county_summary, 5))


Overall LBW Rate: 78.8%
# A tibble: 7 × 3
  age_group n_births lbw_rate
  <fct>        <int>    <dbl>
1 <20         100062     82.7
2 20-24       190245     76.4
3 25-29       199195     77.4
4 30-34       164724     78.9
5 35-39       101766     79.1
6 40+          36510     86.7
7 NA              24    100  
# A tibble: 5 × 4
  race_of_mother                     n_births lbw_rate percentage
  <chr>                                 <int>    <dbl>      <dbl>
1 Native American                        2717     96.9      0.343
2 Unknown/Not Reported                  48924     92.5      6.17 
3 Asian or Pacific Islander/Hawaiian    48829     89.7      6.16 
4 White                                551353     80.7     69.6  
5 African American (Black)             140703     62.2     17.8  

Top 5 Counties with Highest LBW Rates:
# A tibble: 5 × 6
  county_of_birth  n_births lbw_rate below_hs unemployment population
  <chr>               <int>    <dbl>    <dbl>        <dbl>      <dbl>
1 Adams Co

In [ ]:
# RANDOM EFFECTS MODELS

# Model 1: Null model (ICC calculation)
model_null <- glmer(low_birth_weight ~ 1 + (1 | county_of_birth),
                    data = ohio_data,
                    family = binomial(link = "logit"),
                    control = glmerControl(optimizer = "bobyqa"))

# Calculate ICC
var_county <- as.data.frame(VarCorr(model_null))$vcov[1]
icc <- var_county / (var_county + (pi^2/3))
cat(sprintf("\nIntraclass Correlation (ICC): %.3f (%.1f%% variance between counties)\n", 
            icc, icc*100))

# Model 2: Maternal-level predictors only
model_maternal <- glmer(low_birth_weight ~ age_of_mother + race_of_mother + 
                        (1 | county_of_birth),
                        data = ohio_data,
                        family = binomial(link = "logit"),
                        control = glmerControl(optimizer = "bobyqa"))

# Model 3: Full model with county-level predictors
model_full <- glmer(low_birth_weight ~ age_of_mother + race_of_mother + 
                    Below.HS.Education.Percentage + Unemployment.Rate +
                    (1 | county_of_birth),
                    data = ohio_data,
                    family = binomial(link = "logit"),
                    control = glmerControl(optimizer = "bobyqa"))
